# Dynamic Time Warping

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sbn
from scipy.spatial.distance import euclidean

import numbers
from collections import defaultdict

import gower
import time

import scipy.io as sio
import pandas as pd

from joblib import Parallel, delayed
import math
import time

from scipy.spatial.distance import euclidean

import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sbn


def dtw_MTS_vFinal(x, y, dist, plotCostMatrix=False):
    x, y = control_inputs(x, y)
    return dtw(x, y, dist, plotCostMatrix)


def control_inputs(x, y):
    x = np.asanyarray(x, dtype='float')
    y = np.asanyarray(y, dtype='float')

    if x.ndim == y.ndim > 1 and x.shape[1] != y.shape[1]:
        raise ValueError('second dimension of x and y must be the same.')

    return x, y


def plotcostmatrix(warp_path, cost):
    fig, ax = plt.subplots(figsize=(12, 10))
    ax = sbn.heatmap(cost, annot=True, square=True, linewidths=0.1, cmap="YlGnBu", ax=ax)

    # Get the warp path in x and y directions
    path_x = [p[0] for p in warp_path]
    path_y = [p[1] for p in warp_path]

    # Align the path from the center of each cell
    path_xx = [x+0.5 for x in path_x]
    path_yy = [y+0.5 for y in path_y]

    ax.plot(path_xx, path_yy, color='blue', linewidth=3, alpha=0.2)


def dtw(x, y, dist, plotCostMatrix):
    len_x, len_y = len(x), len(y)
    
    window = [(i, j) for i in range(len_x) for j in range(len_y)]
    window = ((i + 1, j + 1) for i, j in window)
    
    dist_matrix = np.zeros((len_x, len_y))
    D = defaultdict(lambda: (float('inf'),))
    D[0, 0] = (0, 0, 0)
    for i, j in window:
        df = pd.DataFrame(np.array([x[i-1], y[j-1]]))
        dt = gower.gower_matrix(df)[1][0]
        D[i, j] = min((D[i-1, j][0]+dt, i-1, j), (D[i, j-1][0]+dt, i, j-1),
                      (D[i-1, j-1][0]+dt, i-1, j-1))


    dtw_distance = D[len_x, len_y][0]
    
    if dist=="onlyDTWdistance":
        return dtw_distance
    else:
        path = []
        i, j = len_x, len_y
        while not (i == j == 0):
            path.append((i, j))
            i, j = D[i, j][1], D[i, j][2]
        path.reverse()

        posiciones = list(D)
        index = 0
        long = list(D.keys())[-1][0]
        cost = np.zeros((long+1,long+1))
        for i in range(long+1):
            for j in range(long+1):
                cost[posiciones[index]] = list(D.values())[index][0]
                index += 1

        if plotCostMatrix:
            plotcostmatrix(path, cost)
            
    return (dtw_distance, path, dist_matrix, cost)

## min-max

In [ ]:
folders = ["S1", "S2", "S3", "S4", "S5"]

for index in range(len(folders)):
    start_time = time.time()
    
    # Load normalized X_train data
    X_train = np.load('../df_to_load/DataToPaperAndTFM_Mod1/types_of_norms/Subconjuntos_3D_norm_minmax/' + folders[index] + '/X_train_tensor.npy')
    print("X_train normalized with MinMax")
    print(X_train.shape)
    
    # Calculate DTW matrix for X_train data
    dtw_matrix_train = Parallel(n_jobs=-1)(delayed(dtw_MTS_vFinal)(X_train[i], X_train[j], "onlyDTWdistance", plotCostMatrix=False) for i in range(X_train.shape[0]) for j in range(X_train.shape[0]))
    data_train = np.array(dtw_matrix_train).reshape((X_train.shape[0], X_train.shape[0]))
    
    print("FINAL DIMENSION TRAIN DTW: ", data_train.shape)
    
    # Save the DTW matrix for X_train
    pd.DataFrame(data_train).to_csv("./data_generated_by_dtw/DTW_D/" + folders[index] + "/X_train.csv", index=False)

    end_time = time.time()
    print("TRAIN Time: ", end_time - start_time)
    
    # Start processing X_test data
    start_time = time.time()
    
    # Load normalized X_test data
    X_test = np.load('../df_to_load/DataToPaperAndTFM_Mod1/types_of_norms/Subconjuntos_3D_norm_minmax/' + folders[index] + '/X_test_tensor.npy')
    print("X_test normalized with MinMax")
    print(X_test.shape)
    
    # Calculate DTW matrix for X_test data
    dtw_matrix_test = Parallel(n_jobs=-1)(delayed(dtw_MTS_vFinal)(X_test[i], X_train[j], "onlyDTWdistance", plotCostMatrix=False) for i in range(X_test.shape[0]) for j in range(X_train.shape[0]))
    data_test = np.array(dtw_matrix_test).reshape((X_test.shape[0], X_train.shape[0]))
    
    print("FINAL DIMENSION TEST DTW: ", data_test.shape)
    
    # Save the DTW matrix for X_test
    pd.DataFrame(data_test).to_csv("./data_generated_by_dtw/DTW_D/" + folders[index] + "/X_test.csv", index=False)
    
    end_time = time.time()
    print("TEST Time: ", end_time - start_time)